# 2.1.5 Algebraic Multigrid Methods

more is coming ...

## The builtin *h1amg*

reduce memory requirements to a minimum:

In [ ]:
from ngsolve import *
from ngsolve.la import EigenValues_Preconditioner
import netgen.meshing
netgen.meshing.Mesh.EnableTableClass("edges", False)
netgen.meshing.Mesh.EnableTableClass("faces", False)

with TaskManager(pajetrace=10**8):
    mesh = Mesh(unit_cube.GenerateMesh(maxh=0.1))
    for l in range(3): mesh.Refine()    

In [ ]:
# fes = H1(mesh, order=1, order_policy=ORDER_POLICY.CONSTANT)  # todo: fix withtout edge/face tables
fes = FESpace("nodal", mesh, order=1)
print ("ndof=", fes.ndof)
u,v = fes.TnT()
a = BilinearForm(grad(u)*grad(v)*dx + 0.1*u*v*dx)
pre = Preconditioner(a, "h1amg")
with TaskManager(pajetrace=10**8):
    a.Assemble();
    lam = EigenValues_Preconditioner(a.mat, pre.mat)
    print (list(lam[0:3]), '...', list(lam[-3:-1]))

## H(curl) - AMG

(WIP)

In [ ]:
from ngsolve import *
from ngsolve.la import EigenValues_Preconditioner
import netgen.meshing
netgen.meshing.Mesh.EnableTableClass("edges", True)
netgen.meshing.Mesh.EnableTableClass("faces", True)

with TaskManager():
    mesh = Mesh(unit_cube.GenerateMesh(maxh=0.1))
    for l in range(1): mesh.Refine()    

In [ ]:
fes = HCurl(mesh, order=0)
print ("ndof = ", fes.ndof)
u,v = fes.TnT()

a = BilinearForm(curl(u)*curl(v)*dx + 0.01*u*v*dx)
pre = Preconditioner(a, "hcurlamg")
with TaskManager(): # pajetrace=10**9):
    a.Assemble();
    lam = EigenValues_Preconditioner(a.mat, pre.mat)
    print (list(lam[0:3]), '...', list(lam[-3:-1]))

In [ ]:
f = LinearForm(curl(v)[2]*dx).Assemble()
gfu = GridFunction(fes)
from ngsolve.krylovspace import CGSolver

inv = CGSolver(a.mat, pre.mat, printrates=True, maxiter=200)

gfu.vec[:] = inv*f.vec